<a href="https://colab.research.google.com/github/Alex-Jung-HB/0715_python_YOLOv8_Youtube/blob/main/0715_python_YOLOv8_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video processing by downloading a Youtube video by Claude

In [4]:
# === 필수 라이브러리 설치 ===
!pip install ultralytics    # YOLO 모델을 위한 Ultralytics 패키지 설치
!pip install yt-dlp         # YouTube 비디오 다운로드를 위한 라이브러리
!pip install opencv-python  # 비디오/이미지 처리를 위한 OpenCV 라이브러리

# === 라이브러리 임포트 ===
from google.colab import files  # Google Colab에서 파일 업로드/다운로드용
from ultralytics import YOLO    # YOLO 객체 탐지 모델
import yt_dlp                   # YouTube 다운로더 (youtube-dl의 개선된 버전)
import cv2                      # OpenCV - 컴퓨터 비전 라이브러리
import os                       # 운영체제 인터페이스 (파일 시스템 작업)
from IPython.display import Video, display  # Jupyter/Colab에서 비디오 표시용

# === YOLO 모델 로드 및 설정 ===
# COCO 데이터셋으로 사전 훈련된 YOLOv8 nano 모델 로드
model = YOLO("yolov8n.pt")
# 모델 크기별 특징:
# - yolov8n.pt: nano (가장 빠름, 가장 가벼움, 정확도 낮음)
# - yolov8s.pt: small (빠름, 가벼움, 중간 정확도)
# - yolov8m.pt: medium (중간 속도, 중간 크기, 좋은 정확도)
# - yolov8l.pt: large (느림, 큰 크기, 높은 정확도)
# - yolov8x.pt: extra large (가장 느림, 가장 큼, 최고 정확도)

# === 모델 정보 출력 ===
model.info()    # YOLOv8 모델의 상세 정보를 출력 (레이어 구조, 파라미터 수 등)
'''
model.info() 출력 정보:
- 모델 구조 (layer-by-layer)
- 총 파라미터 수
- 모델 크기 (MB)
- FLOPs (연산량)
- 지원하는 클래스 수 (COCO: 80개 클래스)

YOLO 모델별 비교:
- nano (n): 3.2M 파라미터, 8.7MB, 빠른 추론
- small (s): 11.2M 파라미터, 22.5MB, 균형잡힌 성능
- medium (m): 25.9M 파라미터, 52.0MB, 높은 정확도
- large (l): 43.7M 파라미터, 87.7MB, 매우 높은 정확도
- extra large (x): 68.2M 파라미터, 136.7MB, 최고 정확도
'''

# === 모델 추가 학습 (선택사항) ===
# 주의: 이미 훈련된 모델을 사용할 경우 이 부분을 생략할 수 있습니다
# 특별한 도메인의 데이터에 대해 fine-tuning하고 싶을 때만 실행
results = model.train(data="coco8.yaml", epochs=10, imgsz=640)
'''
학습 파라미터 상세 설명:

data="coco8.yaml":
  - COCO8은 COCO 데이터셋의 축소 버전 (테스트/데모용)
  - 8개의 샘플 이미지만 포함
  - 80개의 객체 클래스 모두 포함 (person, car, dog, cat 등)
  - 실제 프로젝트에서는 "coco.yaml" (전체 데이터셋) 사용 권장
  - 커스텀 데이터셋 사용 시 자체 .yaml 파일 생성 필요

epochs (에포크): 전체 데이터셋을 몇 번 반복 학습할지 결정
  - epochs=1:   매우 빠른 테스트 (성능 매우 낮음)
  - epochs=10:  빠른 테스트/데모용 (현재 설정)
  - epochs=50:  일반적인 실험용
  - epochs=100: 실제 프로젝트 최소 권장값
  - epochs=300: 고성능 모델을 위한 충분한 학습
  - epochs=500+: 최고 성능을 위한 장시간 학습

imgsz (이미지 크기): 학습/추론 시 사용할 이미지 크기 (픽셀)
  - imgsz=320:  매우 빠름, 낮은 정확도, 작은 객체 탐지 어려움
  - imgsz=640:  표준 크기 (현재 설정), 속도와 정확도의 균형
  - imgsz=1280: 느리지만 높은 정확도, 작은 객체도 잘 탐지
  - 주의: 큰 이미지 크기는 GPU 메모리를 많이 사용함

추가 학습 파라미터들:
  - batch=16: 배치 크기 (한번에 처리할 이미지 수)
  - lr0=0.01: 초기 학습률
  - weight_decay=0.0005: 가중치 감소 (과적합 방지)
  - mosaic=1.0: 모자이크 증강 확률
  - device=0: GPU 디바이스 번호 (0번 GPU 사용)
  - workers=8: 데이터 로딩용 워커 프로세스 수
'''

# YouTube 비디오 다운로드 함수
def download_youtube_video(url, output_path='./'):
    """
    def: This is the Python keyword that means "I'm creating a function"
    download_youtube_video: a name of the function (like giving it a name tag)
    url: first input parameter. YouTube video link
    output_path='./' -the second input parameter(the folder where you want to save the video),
    ='./': if you don't specify a folder, it will use ./ (current folder) by default
    : (Colon): tells Python "the function code starts on the next line"
    """
    # yt-dlp 다운로드 옵션 설정: yt-dlp:YouTube Downloader Plus(Python tool/library that downloads videos from YouTube and many other websites)
    ydl_opts = {    # ydl_opts = YouTube DownLoader options(It's a dictionary that tells the YouTube downloader how to behave)
        'format': 'best[height<=720][ext=mp4]/best[height<=720]/best[ext=mp4]/best',  # 720p+mp4 우선/차선으로 720p+다른 포맷/차차선으로 여타해상도+mp4/차차차선으로 가능한 최대해상도 파일
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),  # outtmpl: 파일명 템플릿, %(title)s = Video's title, %(ext)s = File extension(.mp4, .webm, etc.)
        'noplaylist': True,  # 플레이리스트가 아닌 단일 비디오만 다운로드
        'extractaudio': False,  # 오디오만 추출하지 않음
        'ignoreerrors': True,  # 일부 오류 무시하고 계속 진행
        # 추가 옵션들:
        # 'format': 'worst' - 최저 품질 (빠른 다운로드)
        # 'format': 'best' - 최고 품질 (큰 용량)
        # 'format': 'mp4' - mp4 포맷만 다운로드
    }

    # yt-dlp 객체 생성 및 다운로드 실행
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            print("비디오 정보를 가져오는 중...")
            # 먼저 비디오 정보만 추출 (다운로드 없이)
            info = ydl.extract_info(url, download=False)    # .extract_info(): A method that gets information about the video, download X

            print(f"제목: {info.get('title', 'Unknown')}")    # If 'title' doesn't exist in info → use 'Unknown' instead
            print(f"길이: {info.get('duration', 0)}초")       # If 'duration' doesn't exist in info → use 0,
            print(f"해상도: {info.get('width', 'Unknown')}x{info.get('height', 'Unknown')}")

            # 실제 다운로드 실행
            print("비디오 다운로드 중...")
            info = ydl.extract_info(url, download=True)

            # 다운로드된 비디오 정보 추출
            video_title = info.get('title', 'video')  # 비디오 제목 (없으면 'video' 기본값)
            video_ext = info.get('ext', 'mp4')        # 비디오 확장자 (없으면 'mp4' 기본값)

            # 파일명에서 특수문자 제거 (파일 시스템 호환성)
            import re   # re = Regular Expressions. A Python library for finding and replacing text patterns
            safe_title = re.sub(r'[<>:"/\\|?*]', '_', video_title)    # re.sub(pattern(what to find), replacemen(what to replace it with)), text(where to search)),
            video_filename = f"{safe_title}.{video_ext}"

            # 다운로드된 파일의 전체 경로 생성
            full_path = os.path.join(output_path, video_filename)   # os.path.join(): Operaing System. A function that combines folder paths and filenames properly, output_path: The folder where you want to save the video

            # 실제 다운로드된 파일 찾기 (제목이 변경될 수 있음)
            expected_path = os.path.join(output_path, f"{video_title}.{video_ext}")
            if os.path.exists(expected_path):
                return expected_path
            elif os.path.exists(full_path):
                return full_path
            else:
                # 디렉토리에서 가장 최근에 생성된 비디오 파일 찾기
                video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.webm']
                files_in_dir = []
                for f in os.listdir(output_path):
                    if any(f.lower().endswith(ext) for ext in video_extensions):
                        file_path = os.path.join(output_path, f)
                        files_in_dir.append((file_path, os.path.getctime(file_path)))

                if files_in_dir:
                    # 가장 최근 파일 반환
                    latest_file = max(files_in_dir, key=lambda x: x[1])[0]
                    print(f"다운로드된 파일 찾음: {latest_file}")
                    return latest_file
                else:
                    print("다운로드된 파일을 찾을 수 없습니다.")
                    return None

        except yt_dlp.DownloadError as e:
            print(f"YouTube 다운로드 오류: {e}")
            print("가능한 원인:")
            print("- 비디오가 비공개이거나 삭제됨")
            print("- 지역 제한이 있는 비디오")
            print("- 연령 제한이 있는 비디오")
            return None
        except Exception as e:
            # 다운로드 실패 시 오류 메시지 출력 및 None 반환
            print(f"다운로드 오류: {e}")
            print("URL을 다시 확인해주세요.")
            return None

# YouTube URL 입력받기
youtube_url = input("YouTube URL을 입력하세요: ")
print("비디오를 다운로드 중...")

# YouTube에서 비디오 다운로드 실행
video_path = download_youtube_video(youtube_url)

# 다운로드 성공 여부 확인
if video_path and os.path.exists(video_path):
    print(f"다운로드 완료: {video_path}")

    # === 비디오 파일 유효성 검사 ===
    test_cap = cv2.VideoCapture(video_path)
    if not test_cap.isOpened():
        print("오류: 비디오 파일을 열 수 없습니다.")
        print("다른 URL을 시도해보세요.")
        exit()

    # 비디오 기본 정보 확인
    test_ret, test_frame = test_cap.read()
    if not test_ret:
        print("오류: 비디오 프레임을 읽을 수 없습니다.")
        test_cap.release()
        exit()

    test_cap.release()
    print("비디오 파일 유효성 검사 통과")

    # === YOLO 객체 탐지 시작 ===
    print("YOLO 객체 탐지를 시작합니다...")

    # 주의: model(video_path)는 전체 비디오를 한번에 처리합니다
    # 큰 비디오의 경우 메모리 부족이 발생할 수 있으므로
    # 아래에서 프레임별로 처리하는 방법을 사용합니다
    # results = model(video_path)   # 전체 비디오 한번에 처리 (메모리 주의)

    # === 비디오 파일 정보 읽기 ===
    output_video_path = "detected_video.mp4"  # 결과 비디오 저장 경로

    # OpenCV VideoCapture 객체 생성 (비디오 읽기용)
    cap = cv2.VideoCapture(video_path)

    # 비디오 속성 정보 추출
    fps = int(cap.get(cv2.CAP_PROP_FPS))                    # 초당 프레임 수 (Frame Per Second)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))          # 비디오 너비 (픽셀)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))        # 비디오 높이 (픽셀)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))   # 총 프레임 수

    # FPS가 0인 경우 기본값 설정 (일부 비디오 파일에서 발생)
    if fps <= 0:
        fps = 30
        print("FPS를 감지할 수 없어 30fps로 설정합니다.")

    print(f"비디오 정보: {width}x{height}, {fps}fps, 총 {total_frames}프레임")

    # === 결과 비디오 저장 설정 ===
    # 비디오 코덱 설정 (mp4v: MP4 비디오 압축 방식)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # VideoWriter 객체 생성 (비디오 쓰기용)
    # 매개변수: (출력파일명, 코덱, fps, (너비, 높이))
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # VideoWriter 초기화 확인
    if not out.isOpened():
        print("오류: 출력 비디오 파일을 생성할 수 없습니다.")
        cap.release()
        exit()

    # === 프레임별 처리 루프 ===
    frame_count = 0  # 현재 처리 중인 프레임 번호

    print("프레임별 객체 탐지 처리 시작...")

    try:
        while True:
            # 비디오에서 한 프레임씩 읽기
            ret, frame = cap.read()

            # ret: 프레임 읽기 성공 여부 (True/False)
            # frame: 읽어온 프레임 이미지 데이터 (numpy 배열)

            if not ret:
                # 더 이상 읽을 프레임이 없으면 루프 종료
                print("모든 프레임 처리 완료")
                break

            # === 현재 프레임에서 YOLO 객체 탐지 실행 ===
            frame_results = model(frame)
            # frame_results: YOLO 탐지 결과 (탐지된 객체들의 좌표, 클래스, 신뢰도 등)

            # === 탐지 결과를 프레임에 시각화 ===
            # plot() 메서드: 바운딩 박스, 라벨, 신뢰도를 프레임에 그려줌
            annotated_frame = frame_results[0].plot()

            # 시각화 옵션들:
            # frame_results[0].plot(show_labels=False)    # 라벨 숨기기
            # frame_results[0].plot(show_conf=False)      # 신뢰도 숨기기
            # frame_results[0].plot(line_width=2)         # 선 두께 조정

            # === 처리된 프레임을 결과 비디오에 저장 ===
            out.write(annotated_frame)

            # === 진행 상황 표시 ===
            frame_count += 1
            if frame_count % 30 == 0:  # 30프레임마다 진행상황 출력 (너무 자주 출력하면 느려짐)
                progress_percent = (frame_count / total_frames) * 100 if total_frames > 0 else 0
                print(f"처리 진행률: {frame_count}/{total_frames} ({progress_percent:.1f}%)")

            # 선택사항: 특정 프레임에서 중단하기 (테스트용)
            # if frame_count > 300:  # 처음 300프레임만 처리
            #     break

    except KeyboardInterrupt:
        print("\n사용자에 의해 처리가 중단되었습니다.")
    except Exception as e:
        print(f"비디오 처리 중 오류 발생: {e}")
    finally:
        # 오류가 발생해도 리소스는 항상 해제
        # === 리소스 정리 및 메모리 해제 ===
        cap.release()           # VideoCapture 객체 해제 (비디오 파일 닫기)
        out.release()           # VideoWriter 객체 해제 (출력 비디오 파일 닫기)
        # cv2.destroyAllWindows() # Google Colab에서는 GUI 지원이 없어 주석 처리

    print(f"객체 탐지 완료! 결과 비디오: {output_video_path}")

    # === 결과 비디오 다운로드 ===
    print("\n=== 처리된 비디오 다운로드 ===")
    try:
        # Google Colab에서 파일 다운로드
        files.download(output_video_path)
        print(f"다운로드 시작: {output_video_path}")
    except Exception as e:
        print(f"다운로드 오류: {e}")
        print("수동으로 파일을 다운로드하려면 파일 탭에서 다운로드하세요.")

    # === 결과 비디오 표시 (Google Colab 전용) ===
    # Colab 환경에서 비디오를 인라인으로 재생
    try:
        display(Video(output_video_path, width=640, height=480))
        print("비디오 재생이 시작됩니다.")
    except Exception as e:
        print(f"비디오 표시 오류: {e}")
        print(f"비디오 파일이 저장되었습니다: {output_video_path}")

    # === 탐지 결과 통계 분석 ===
    print("\n=== 탐지 결과 요약 ===")

    # 마지막 프레임의 탐지 결과를 기반으로 통계 생성
    # 주의: 전체 비디오 통계가 아닌 마지막 프레임 기준
    # 전체 비디오 통계를 원한다면 루프 내에서 수집해야 함

    # 마지막 프레임에서 한번 더 탐지 수행 (통계용)
    cap_stats = cv2.VideoCapture(video_path)
    ret, sample_frame = cap_stats.read()
    if ret:
        sample_results = model(sample_frame)
        result = sample_results[0]

        # 탐지된 객체가 있는지 확인
        if hasattr(result, 'boxes') and result.boxes is not None and len(result.boxes) > 0:
            # 탐지된 클래스들의 고유값 추출
            detected_classes = result.boxes.cls.unique()

            # 클래스 ID를 클래스 이름으로 변환
            class_names = [model.names[int(cls)] for cls in detected_classes]

            # 각 클래스별 탐지 개수 계산
            class_counts = {}
            for cls in result.boxes.cls:
                class_name = model.names[int(cls)]
                class_counts[class_name] = class_counts.get(class_name, 0) + 1

            # 결과 출력
            print(f"탐지된 클래스: {class_names}")
            print(f"총 탐지된 객체 수: {len(result.boxes)}")
            print("클래스별 개수:")
            for class_name, count in class_counts.items():
                print(f"  - {class_name}: {count}개")

            # 신뢰도 통계
            confidences = result.boxes.conf.cpu().numpy()  # GPU에서 CPU로 이동 후 numpy 변환
            print(f"평균 신뢰도: {confidences.mean():.3f}")
            print(f"최고 신뢰도: {confidences.max():.3f}")
            print(f"최저 신뢰도: {confidences.min():.3f}")
        else:
            print("샘플 프레임에서 탐지된 객체가 없습니다.")

    cap_stats.release()

else:
    # 다운로드 실패 시 오류 메시지
    print("비디오 다운로드에 실패했습니다. 다음을 확인해주세요:")
    print("1. YouTube URL이 올바른지 확인")
    print("2. 비디오가 비공개/삭제되지 않았는지 확인")
    print("3. 인터넷 연결 상태 확인")
    print("4. 비디오 연령 제한이나 지역 제한이 없는지 확인")

# === 파일 정리 함수 ===
def cleanup_files():
    """
    처리 후 임시 파일들을 정리합니다.
    대용량 비디오 파일들이 디스크 공간을 차지하지 않도록 삭제
    """
    files_to_remove = []

    # 원본 다운로드 파일 확인
    if 'video_path' in globals() and video_path and os.path.exists(video_path):
        files_to_remove.append(video_path)

    # 결과 비디오 파일 확인 (선택적 삭제)
    if os.path.exists("detected_video.mp4"):
        # files_to_remove.append("detected_video.mp4")  # 주석 해제 시 결과 파일도 삭제
        pass

    # 파일 삭제 실행
    for file_path in files_to_remove:
        try:
            os.remove(file_path)
            print(f"파일 삭제됨: {file_path}")
        except Exception as e:
            print(f"파일 삭제 실패 {file_path}: {e}")

# === 추가 유틸리티 함수들 ===

def get_video_info(video_path):
    """비디오 파일의 상세 정보를 출력합니다."""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps

    print(f"=== 비디오 정보 ===")
    print(f"해상도: {width} x {height}")
    print(f"FPS: {fps}")
    print(f"총 프레임: {frame_count}")
    print(f"재생 시간: {duration:.2f}초 ({duration/60:.1f}분)")

    cap.release()
    return {
        'width': width, 'height': height, 'fps': fps,
        'frame_count': frame_count, 'duration': duration
    }

def process_specific_timerange(video_path, start_sec, end_sec):
    """
    비디오의 특정 시간 구간만 처리합니다.
    start_sec: 시작 시간 (초)
    end_sec: 종료 시간 (초)
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # 시작 프레임으로 이동

    frame_count = start_frame
    while frame_count < end_frame:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 처리
        results = model(frame)
        annotated_frame = results[0].plot()

        # 여기서 프레임 저장 또는 다른 처리 수행

        frame_count += 1

    cap.release()

# 파일 정리 실행 (필요시 주석 해제)
# cleanup_files()

print("\n=== 처리 완료 ===")
print("결과 비디오 파일: detected_video.mp4")
print("필요시 cleanup_files() 함수를 호출하여 임시 파일을 정리하세요.")

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

train: Scanning /content/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 853.5±177.7 MB/s, size: 54.0 KB)



val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

Plotting labels to runs/detect/train3/labels.jpg... 


...

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

0: 384x640 11 cars, 1 bus, 2 trucks, 105.1ms
Speed: 1.9ms preprocess, 105.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 bus, 2 trucks, 95.6ms
Speed: 1.6ms preprocess, 95.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 3 trucks, 102.8ms
Speed: 1.6ms preprocess, 102.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 3 trucks, 102.3ms
Speed: 1.5ms preprocess, 102.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 2 trucks, 100.4ms
Speed: 1.5ms preprocess, 100.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 2 trucks, 101.6ms
Speed: 1.9ms preprocess, 101.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 102.2ms
Speed: 1.7ms preprocess, 102.2ms inference, 1.0ms postprocess per image

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

다운로드 시작: detected_video.mp4


비디오 재생이 시작됩니다.

=== 탐지 결과 요약 ===

0: 384x640 1 person, 15 cars, 1 bus, 2 trucks, 114.7ms
Speed: 1.7ms preprocess, 114.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
탐지된 클래스: ['person', 'car', 'bus', 'truck']
총 탐지된 객체 수: 19
클래스별 개수:
  - car: 15개
  - truck: 2개
  - person: 1개
  - bus: 1개
평균 신뢰도: 0.665
최고 신뢰도: 0.864
최저 신뢰도: 0.254

=== 처리 완료 ===
결과 비디오 파일: detected_video.mp4
필요시 cleanup_files() 함수를 호출하여 임시 파일을 정리하세요.


Video processing by uploading a video from PC

In [9]:
!pip install ultralytics opencv-python    # pip:the standard package installer for Python

from google.colab import files
from ultralytics import YOLO
import cv2
import os

# COCO 사전 훈련된 YOLOv8n 모델 로드
model = YOLO("yolov8n.pt")

# 모델 정보 표시 (선택사항)
model.info()    # YOLOv8 모델의 상세 정보를 출력
'''
model_n.info(): 가장 가벼운 모델
model_s.info(): 중간 크기
model_m.info(): 더 큰 모델
'''

# 모델 훈련 (선택사항 - 기본 모델로도 충분할 수 있음)
results = model.train(data="coco8.yaml", epochs=10, imgsz=640)    # coco8의 데이터를 이용하여 학습한 결과 저장
'''
data="coco8.yaml": COCO8은 COCO 데이터셋의 축소 버전
  - 8개 이미지만 포함(테스트용)
  - 80개 클래스 모두 포함
  - 빠른 테스트/데모용)
epochs: 전체 데이터셋을 한 번 학습하는 단위
  - epochs=1   매우 빠름, 성능 낮음
  - epochs=10  테스트용 (현재 설정)
  - epochs=100 일반적인 학습
  - epochs=300 고성능 학습
imgsz=640: 입력 이미지 크기 (픽셀)
  - imgsz=320  빠름, 정확도 낮음
  - imgsz=640  표준 (현재 설정)
  - imgsz=1280 느림, 정확도 높음
'''

# 비디오 파일 업로드
print("비디오 파일을 업로드하세요 (mp4, avi, mov 등):")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]   # dict_keys를 리스트로 변환

# 출력 비디오 파일명 설정
output_path = f"processed_{video_path}"

# 비디오 처리
print("비디오 처리 중...")

# 비디오 파일 열기
cap = cv2.VideoCapture(video_path)

# 비디오 속성 가져오기
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"비디오 정보: {width}x{height}, {fps}fps, 총 {total_frames}프레임")

# 비디오 라이터 설정 (MP4 코덱 사용)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"처리 중: {frame_count}/{total_frames} 프레임", end='\r')

    # YOLOv8로 객체 탐지 수행
    results = model(frame)

    # 결과를 프레임에 그리기
    annotated_frame = results[0].plot()

    # 처리된 프레임을 출력 비디오에 쓰기
    out.write(annotated_frame)

# 리소스 해제
cap.release()
out.release()

print(f"\n비디오 처리 완료! 출력 파일: {output_path}")

# 처리된 비디오 다운로드
print("처리된 비디오를 다운로드합니다...")
files.download(output_path)

print("완료!")

# 파일 정리 (선택사항)
# os.remove(video_path)  # 원본 비디오 삭제
# os.remove(output_path)  # 처리된 비디오 삭제

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

train: Scanning /content/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1026.7±481.0 MB/s, size: 54.0 KB)



val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

Plotting labels to runs/detect/train4/labels.jpg... 


...

                   all          4         17      0.633      0.871      0.888      0.624
                person          3         10      0.696        0.5       0.52      0.279
                   dog          1          1      0.398          1      0.995      0.597
                 horse          1          2      0.749          1      0.995      0.598
              elephant          1          2      0.574      0.723      0.828      0.381
              umbrella          1          1      0.566          1      0.995      0.995
          potted plant          1          1      0.817          1      0.995      0.895
Speed: 1.3ms preprocess, 172.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train4
비디오 파일을 업로드하세요 (mp4, avi, mov 등):


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0: 288x640 8 cars, 1 truck, 90.9ms
Speed: 2.4ms preprocess, 90.9ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 197/1861 프레임
0: 288x640 8 cars, 1 truck, 78.9ms
Speed: 2.3ms preprocess, 78.9ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 198/1861 프레임
0: 288x640 8 cars, 1 truck, 79.4ms
Speed: 5.2ms preprocess, 79.4ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 199/1861 프레임
0: 288x640 10 cars, 1 bus, 1 truck, 80.6ms
Speed: 2.1ms preprocess, 80.6ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 200/1861 프레임
0: 288x640 8 cars, 1 truck, 84.2ms
Speed: 2.7ms preprocess, 84.2ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 201/1861 프레임
0: 288x640 8 cars, 1 truck, 79.1ms
Speed: 2.5ms preprocess, 79.1ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)
처리 중: 202/1861 프레임
0: 288x640 11 cars, 1 truck, 77.6ms
Speed: 2.2ms p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

완료!
